In [1]:
import pandas as pd

In [2]:
dt = pd.read_csv('data/booksummaries_embeddings_0_12839.csv')

In [3]:
dt.head()

,Wikipedia article ID,Freebase ID,Book title,Author,Publication date,Book genres (Freebase ID:name tuples),Plot summary,Summarized Plot Summary,Parsed Genres,Parsed Genres Embedding,Summarized Plot Embedding
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca...","Old Major, an old boar, inspires a farm animal...","Roman à clef, Satire, Children's literature, S...","[-0.0268506221473217, 0.01663643680512905, -0....","[-0.03497184067964554, -0.01884828694164753, -..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan...","In a near-future England, teenager Alex leads ...","Science Fiction, Novella, Speculative fiction,...","[-0.022291284054517746, 0.01301165483891964, -...","[-0.007798242848366499, 0.022935163229703903, ..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...,"In Oran, a plague begins with rats dying in th...","Existentialism, Fiction, Absurdist fiction, Novel","[-0.006543333642184734, 0.014447270892560482, ...","[-0.030361512675881386, -0.01121631357818842, ..."
3,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...,The novel explores a galaxy divided into Zones...,"Hard science fiction, Science Fiction, Specula...","[-0.008878524415194988, 0.024636436253786087, ...","[-0.027841700240969658, -0.06042584404349327, ..."
4,2152,/m/0x5g,All Quiet on the Western Front,Erich Maria Remarque,1929-01-29,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","The book tells the story of Paul Bäumer, a Ge...","Paul Bäumer, a young German soldier in WWI, jo...","War novel, Roman à clef","[-0.03799491003155708, -0.03112509287893772, -...","[-0.02093232050538063, 0.009090326726436615, -..."


In [16]:
from scipy.spatial.distance import cosine
import numpy as np
# 코사인 유사도를 계산하여 가장 비슷한 책 3권 추천 함수
def recommend_books(book_title, top_n=3):
    # 선택한 책의 데이터 가져오기
    selected_books = dt[dt['Book title'] == book_title]
    
    if selected_books.empty:  # 책 제목이 없는 경우 처리
        print(f"Error: Book title '{book_title}' not found in the dataset.")
        return []

    selected_book = selected_books.iloc[0]

    # 선택한 책의 장르 임베딩 추출
    selected_embedding = eval(selected_book['Parsed Genres Embedding'])

    # 코사인 유사도를 저장할 리스트
    similarities = []

    for _, row in dt.iterrows():
        if row['Book title'] != book_title:  # 자기 자신은 제외
            other_embedding = eval(row['Parsed Genres Embedding'])
            similarity = 1 - cosine(selected_embedding, other_embedding)  # 코사인 유사도 계산
            similarities.append((row['Book title'], similarity))

    # 유사도를 기준으로 정렬하여 상위 n개 선택
    top_books = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]

    # 추천 결과 반환
    return top_books


In [17]:
# 예제: 'Animal Farm'과 가장 비슷한 책 3권 추천
book_title = 'Animal Farm'
recommendations = recommend_books(book_title)

print(f"Books similar to '{book_title}':")
for title, similarity in recommendations:
    print(f"{title} (Similarity: {similarity:.4f})")


Books similar to 'Animal Farm':
The Sun Also Rises (Similarity: 0.8792)
The Great Indian Novel (Similarity: 0.8519)
The Bell Jar (Similarity: 0.8174)


 동물 농장의 주요 장르
"Animal Farm"은 주로 아래 장르를 포함합니다:

풍자 (Satire)
정치적 우화 (Political allegory)
디스토피아 (Dystopia)
The Sun Also Rises:
장르: 전후 세대 (Post-War), 실존주의 (Existentialism), 현대 소설 (Modernism).
The Great Indian Novel:
장르: 풍자, 역사적 우화.
The Bell Jar:
장르: 현대 소설, 자전적 소설, 심리 드라마.


In [25]:
# '1984'과 유사한 제목 확인 (부분 문자열 검색)
similar_titles = dt[dt['Book title'].str.contains('brave new world', case=False, na=False)]
print(similar_titles)

    Wikipedia article ID Freebase ID       Book title         Author  \
79                 36681    /m/0952p  Brave New World  Aldous Huxley   

   Publication date              Book genres (Freebase ID:name tuples)  \
79              NaN  {"/m/06n90": "Science Fiction", "/m/0dwly": "C...   

                                         Plot summary  \
79   The novel opens in London in 632 (AD 2540 in ...   

                              Summarized Plot Summary  \
79  In a dystopian future, the World State control...   

                                        Parsed Genres  \
79  Science Fiction, Children's literature, Specul...   

                              Parsed Genres Embedding  \
79  [-0.013131619431078434, 0.025082388892769814, ...   

                            Summarized Plot Embedding  
79  [0.004377966281026602, -0.035789597779512405, ...  


In [26]:
# 예제: 'Brave New World'과 가장 비슷한 책 3권 추천
book_title = 'Brave New World'
recommendations = recommend_books(book_title)

print(f"Books similar to '{book_title}':")
for title, similarity in recommendations:
    print(f"{title} (Similarity: {similarity:.4f})")


Books similar to 'Brave New World':
Fahrenheit 451 (Similarity: 1.0000)
Gathering Blue (Similarity: 0.9826)
The Guardians (Similarity: 0.9656)


Brave New World의 주요 장르와 맥락
장르: 디스토피아(Dystopia), 과학 소설(Science Fiction), 사회 풍자(Social Satire)
Fahrenheit 451:

장르: 디스토피아, 과학 소설.
주제: 지식과 표현의 자유를 억압하는 사회에서의 저항과 자유의 의미.

Gathering Blue:

장르: 디스토피아, 청소년 소설(Young Adult Fiction).
주제: 불완전한 사회에서 예술과 창의성의 역할.

The Guardians:

장르: 디스토피아, 정치 스릴러(Political Thriller).
주제: 권위주의적 통제와 개인 자유의 갈등.


In [27]:
# 코사인 유사도를 계산하여 가장 비슷한 책 3권 추천 함수 (줄거리 임베딩 기반)
def recommend_books_by_plot(book_title, top_n=3):
    # 선택한 책의 데이터 가져오기
    selected_books = dt[dt['Book title'] == book_title]
    
    if selected_books.empty:  # 책 제목이 없는 경우 처리
        print(f"Error: Book title '{book_title}' not found in the dataset.")
        return []

    selected_book = selected_books.iloc[0]

    # 선택한 책의 줄거리 임베딩 추출
    selected_embedding = eval(selected_book['Summarized Plot Embedding'])

    # 코사인 유사도를 저장할 리스트
    similarities = []

    for _, row in dt.iterrows():
        if row['Book title'] != book_title:  # 자기 자신은 제외
            other_embedding = eval(row['Summarized Plot Embedding'])
            similarity = 1 - cosine(selected_embedding, other_embedding)  # 코사인 유사도 계산
            similarities.append((row['Book title'], similarity))

    # 유사도를 기준으로 정렬하여 상위 n개 선택
    top_books = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]

    # 추천 결과 반환
    return top_books

In [28]:
# 예제: 'Brave New World'과 가장 비슷한 책 3권 추천 (줄거리 기반)
book_title = 'Brave New World'
recommendations = recommend_books_by_plot(book_title)

print(f"Books similar to '{book_title}':")
for title, similarity in recommendations:
    print(f"{title} (Similarity: {similarity:.4f})")

Books similar to 'Brave New World':
We (Similarity: 0.6200)
Anthem (Similarity: 0.6124)
Nineteen Eighty-Four (Similarity: 0.6012)


We:

장르: 디스토피아, 정치적 풍자.
주제: 전체주의 사회에서 개인의 자유와 감정의 억압.
유사점: 권위주의적 사회와 통제된 개인의 삶을 묘사하며, "Brave New World"의 철학적 기조와 유사.
평가: 매우 적합한 추천.
Anthem:

장르: 디스토피아, 철학적 소설.
주제: 개인주의와 자유의 가치를 전체주의 사회에서 탐구.
유사점: 전체주의와 자유의 갈등, 개인의 자아 발견이라는 테마가 "Brave New World"와 유사.
평가: 적합한 추천.
Nineteen Eighty-Four:

장르: 디스토피아, 정치 소설.
주제: 권위주의 정권에 의해 통제되는 개인의 삶과 자유.
유사점: 디스토피아적 세계관과 사회 통제라는 큰 틀이 "Brave New World"와 일치.
평가: 적합한 추천.